# Importing the libraries

In [25]:
import evalml
from evalml import AutoMLSearch
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [9]:
# # Load the Iris dataset
# iris = load_iris()
# # iris
# X, y = iris.data, iris.target
data=sns.load_dataset('iris')
data


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [10]:
data['species'].value_counts()

setosa        50
versicolor    50
virginica     50
Name: species, dtype: int64

In [11]:
## Split the data into training and testing sets
X, y = data.drop(columns=['species']), data['species']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
X_train

,sepal_length,sepal_width,petal_length,petal_width
22,4.6,3.6,1.0,0.2
15,5.7,4.4,1.5,0.4
65,6.7,3.1,4.4,1.4
11,4.8,3.4,1.6,0.2
42,4.4,3.2,1.3,0.2
...,...,...,...,...
71,6.1,2.8,4.0,1.3
106,4.9,2.5,4.5,1.7
14,5.8,4.0,1.2,0.2
92,5.8,2.6,4.0,1.2


In [13]:
# Specify the objective to optimize (e.g., 'log loss' for multiclass classification)
objective = 'log loss multiclass'

# Create and run the AutoML search with hyperparameter tuning
automl = AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='multiclass', objective=objective, max_iterations=20)
automl.search()


	High coefficient of variation (cv >= 0.5) within cross validation scores.
	Mode Baseline Multiclass Classification Pipeline may not perform as estimated on unseen data.


{1: {'Random Forest Classifier w/ Label Encoder + Imputer + RF Classifier Select From Model': 2.130824327468872,
  'Total time of batch': 2.8214614391326904},
 2: {'LightGBM Classifier w/ Label Encoder + Imputer + Select Columns Transformer': 0.7162966728210449,
  'Extra Trees Classifier w/ Label Encoder + Imputer + Select Columns Transformer': 1.1966769695281982,
  'Elastic Net Classifier w/ Label Encoder + Imputer + Standard Scaler + Select Columns Transformer': 0.4893350601196289,
  'XGBoost Classifier w/ Label Encoder + Imputer + Select Columns Transformer': 0.8897912502288818,
  'Logistic Regression Classifier w/ Label Encoder + Imputer + Standard Scaler + Select Columns Transformer': 8.373318910598755,
  'Total time of batch': 15.59993314743042},
 3: {'Extra Trees Classifier w/ Label Encoder + Imputer + Select Columns Transformer': 1.2509610652923584,
  'Random Forest Classifier w/ Label Encoder + Imputer + Select Columns Transformer': 5.538467645645142,
  'Elastic Net Classifier

In [14]:
# Access the ranking of pipelines
pipeline_rankings = automl.rankings
pipeline_rankings

,id,pipeline_name,search_order,ranking_score,mean_cv_score,standard_deviation_cv_score,percent_better_than_baseline,high_variance_cv,parameters
0,9,Elastic Net Classifier w/ Label Encoder + Impu...,9,0.107178,0.107178,0.024966,99.548320,False,"{'Label Encoder': {'positive_label': None}, 'I..."
2,3,Extra Trees Classifier w/ Label Encoder + Impu...,3,0.133809,0.133809,0.023493,99.436090,False,"{'Label Encoder': {'positive_label': None}, 'I..."
7,17,Random Forest Classifier w/ Label Encoder + Im...,17,0.146581,0.146581,0.056838,99.382264,False,"{'Label Encoder': {'positive_label': None}, 'I..."
14,1,Random Forest Classifier w/ Label Encoder + Im...,1,0.180150,0.180150,0.101761,99.240793,False,"{'Label Encoder': {'positive_label': None}, 'I..."
16,6,Logistic Regression Classifier w/ Label Encode...,6,0.206716,0.206716,0.025983,99.128839,False,"{'Label Encoder': {'positive_label': None}, 'I..."
17,2,LightGBM Classifier w/ Label Encoder + Imputer...,2,0.227084,0.227084,0.106245,99.043002,False,"{'Label Encoder': {'positive_label': None}, 'I..."
18,5,XGBoost Classifier w/ Label Encoder + Imputer ...,5,0.265383,0.265383,0.032913,98.881598,False,"{'Label Encoder': {'positive_label': None}, 'I..."
19,0,Mode Baseline Multiclass Classification Pipeline,0,23.728738,23.728738,0.520245,0.000000,True,"{'Label Encoder': {'positive_label': None}, 'B..."


In [15]:
# Get the best pipeline
best_pipeline = automl.best_pipeline
best_pipeline

pipeline = MulticlassClassificationPipeline(component_graph={'Label Encoder': ['Label Encoder', 'X', 'y'], 'Imputer': ['Imputer', 'X', 'Label Encoder.y'], 'Standard Scaler': ['Standard Scaler', 'Imputer.x', 'Label Encoder.y'], 'Select Columns Transformer': ['Select Columns Transformer', 'Standard Scaler.x', 'Label Encoder.y'], 'Elastic Net Classifier': ['Elastic Net Classifier', 'Select Columns Transformer.x', 'Label Encoder.y']}, parameters={'Label Encoder':{'positive_label': None}, 'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'knn', 'boolean_impute_strategy': 'most_frequent', 'categorical_fill_value': None, 'numeric_fill_value': None, 'boolean_fill_value': None}, 'Select Columns Transformer':{'columns': ['petal_length', 'petal_width']}, 'Elastic Net Classifier':{'penalty': 'elasticnet', 'C': 8.474044870453413, 'l1_ratio': 0.6235636967859725, 'n_jobs': -1, 'multi_class': 'auto', 'solver': 'saga'}}, random_seed=0)

In [16]:
# Retrieve the best pipeline and summarize the results
best_pipeline = automl.best_pipeline
automl.describe_pipeline(automl.rankings.iloc[0]["id"])


****************************************************************************************************
* Elastic Net Classifier w/ Label Encoder + Imputer + Standard Scaler + Select Columns Transformer *
****************************************************************************************************

Problem Type: multiclass
Model Family: Linear

Pipeline Steps
1. Label Encoder
	 * positive_label : None
2. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : knn
	 * boolean_impute_strategy : most_frequent
	 * categorical_fill_value : None
	 * numeric_fill_value : None
	 * boolean_fill_value : None
3. Standard Scaler
4. Select Columns Transformer
	 * columns : ['petal_length', 'petal_width']
5. Elastic Net Classifier
	 * penalty : elasticnet
	 * C : 8.474044870453413
	 * l1_ratio : 0.6235636967859725
	 * n_jobs : -1
	 * multi_class : auto
	 * solver : saga

Training
Training for multiclass problems.
Total training time (including CV): 0.5 seconds

Cro

# Save the best pipeline


In [17]:
# Save the best pipeline
best_pipeline.save("Multi_Cassification_model.pkl")

# Loading the saved model in order to do prediction

In [18]:
final_model=automl.load('Multi_Cassification_model.pkl')

In [19]:
# Make predictions on the test set
y_pred = final_model.predict(X_test)
y_pred


73     versicolor
18         setosa
118     virginica
78     versicolor
76     versicolor
31         setosa
64     versicolor
141     virginica
68     versicolor
82     versicolor
110     virginica
12         setosa
36         setosa
9          setosa
19         setosa
56     versicolor
104     virginica
69     versicolor
55     versicolor
132     virginica
29         setosa
127     virginica
26         setosa
128     virginica
131     virginica
145     virginica
108     virginica
143     virginica
45         setosa
30         setosa
Name: species, dtype: category
Categories (3, object): ['setosa', 'versicolor', 'virginica']

In [27]:
df_pred=pd.DataFrame({'True_result': y_test, 'Predicted_result':y_pred})
df_pred['Correct_result']=np.where(df_pred['True_result']==df_pred['Predicted_result'],1,0)
df_pred

,True_result,Predicted_result,Correct_result
73,versicolor,versicolor,1
18,setosa,setosa,1
118,virginica,virginica,1
78,versicolor,versicolor,1
76,versicolor,versicolor,1
31,setosa,setosa,1
64,versicolor,versicolor,1
141,virginica,virginica,1
68,versicolor,versicolor,1
82,versicolor,versicolor,1


In [28]:
df_pred['Correct_result'].value_counts()

1    30
Name: Correct_result, dtype: int64

In [21]:

# Evaluate the predictions
classification_report = best_pipeline.score(X_test, y_test, objectives=['log loss multiclass', 'accuracy'])
# print(f"Classification Report:\n{classification_report}")
print(classification_report)


ObjectiveNotFoundError: accuracy is not a valid Objective! Use evalml.objectives.get_all_objective_names() to get a list of all valid objective names. 